# Cahn-Hilliard 1D
Adapted from fenics [Cahn-Hilliard 2D example](https://fenicsproject.org/docs/dolfin/latest/python/demos/cahn-hilliard/demo_cahn-hilliard.py.html).

First form taken from paper: [_High Accuracy Benchmark Problems for Allen-Cahn and Cahn-Hilliard Dynamics_](http://www.global-sci.com/intro/article_detail/cicp/13225.html):
$$u_t = -\epsilon^2\Delta\Delta u +\Delta(W^\prime(u)),\tag{1}$$where $$W(u) = \frac{1}{4}(u^2-1)^2$$
Second form taken from paper: [_Solving Allen-Cahn and Cahn-Hilliard Equations using the Adaptive Physics Informed Neural Networks_](https://arxiv.org/abs/2007.04542)
$$u_t = \Delta(\gamma_2(u^3-u)-\gamma_1\Delta u) \tag{2}$$

In [1]:
from dolfin import *
from time import perf_counter
import scipy.io
import numpy as np
import matplotlib.animation as animation
import matplotlib.pyplot as plt
%matplotlib notebook

output = "solutions/cahn-hilliard-1d/"

# initial conditions
RANDOM_ICs = 1
EXPR_ICs   = 2

In [2]:
# parameters
#####################################

nx       = 1000        # mesh points
theta    = 0.5         # combo of current solution with previous
dt       = 5.0e-3      # time step
(x0, xf) = (0, 2*pi)   # x-boundaries

IC       = EXPR_ICs    # initial conditions
epsilon  = 0.18        # for EXPR_ICs
height   = 1           # for RANDOM_ICs

# Form compiler options
parameters["form_compiler"]["optimize"]     = True
parameters["form_compiler"]["cpp_optimize"] = True

In [3]:
# Class for interfacing with the Newton solver
class CahnHilliardEquation(NonlinearProblem):
    def __init__(self, a, L):
        NonlinearProblem.__init__(self)
        self.L = L
        self.a = a
    def F(self, b, x):
        assemble(self.L, tensor=b)
    def J(self, A, x):
        assemble(self.a, tensor=A)

In [4]:
class PeriodicBoundary(SubDomain):
    '''Sub domain for Periodic boundary condition'''
    # Left boundary is target domain
    def inside(self, x, on_boundary):
        return near(x[0], x0) and on_boundary

    # Map right boundary to left boundary
    def map(self, x, y):
        y[0] = x[0] - (xf - x0)

In [5]:
class RandomIC(UserExpression):
    '''Randomized uniform distribution over [0, height]'''
    def __init__(self, height, **kwargs):
        np.random.seed(2 + MPI.rank(MPI.comm_world))
        super().__init__(**kwargs)
        self.height = height
    def eval(self, values, x):
        values[0] = self.height * np.random.rand()
        values[1] = 0.0
    def value_shape(self):
        return (2,)

### _Benchmark III_
Initial conditions taken from paper: [_High Accuracy Benchmark Problems for Allen-Cahn and Cahn-Hilliard Dynamics_](http://www.global-sci.com/intro/article_detail/cicp/13225.html).
$$u(x,0)=cos(2x)+\frac{1}{100}e^{\cos(x+\frac{1}{10})} $$

In [6]:
class ExprIC(UserExpression):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
    def eval(self, values, x):
        values[0] = np.cos(2*x[0]) + 1/100 * np.exp(np.cos(x[0] + 1/10))
        values[1] = 0.0
    def value_shape(self):
        return (2,)

In [7]:
# formulate problem
#####################################

# create periodic boundary condition
pbc = PeriodicBoundary(x0, xf)
bc = []

# setup mesh
mesh = IntervalMesh(nx, x0, xf)
P1 = FiniteElement("Lagrange", mesh.ufl_cell(), 1)
ME = FunctionSpace(mesh, P1*P1, constrained_domain=pbc)

# define trial and test functions
du = TrialFunction(ME)
(v, q) = TestFunction(ME)

# define functions
u = Function(ME)
u0 = Function(ME)

# split mixed functions
(dq, dmu) = split(du)
(c,  mu)  = split(u)
(c0, mu0) = split(u0)

# initial conditions
if IC == RANDOM_ICs:
    u_init = RandomIC(height, degree=0)
elif IC == EXPR_ICs:
    u_init = ExprIC(degree=1)
u.interpolate(u_init)
u0.interpolate(u_init)

# time discretization
# mu_(n+theta)
mu_mid = (1.0-theta)*mu0 + theta*mu

TypeError: __init__(): incompatible constructor arguments. The following argument types are supported:
    1. dolfin.cpp.mesh.SubDomain(map_tol: float = 1e-10)

Invoked with: 0, 6.283185307179586

In [ ]:

c = variable(c)
W = 1/4 * (c**2 - 1)**2
dWdc = diff(W, c)

F0 = c*q*dx - c0*q*dx + dt*epsilon**2*dot(grad(mu_mid), grad(q))*dx + \
      dt*dot(grad(dWdc), grad(q))*dx
F1 = mu*v*dx - dot(grad(c), grad(v))*dx
F = F0 + F1

# gamma1 = Constant(1.0e-6)
# gamma2 = Constant(1.0/100)

# F0 = c*q*dx - c0*q*dx + dt*dot(grad(mu_mid), grad(q))*dx
# F1 = mu*v*dx - gamma2*(c**3 - c)*v*dx - gamma1*dot(grad(c), grad(v))*dx
# F = F0 + F1

# compute Jacobian
J = derivative(F, u, du)

In [ ]:
# problem for Newton Solver
# problem = CahnHilliardEquation(J, F)
# solver = NewtonSolver()

# problem for PETScSolver
problem = NonlinearVariationalProblem(F, u, bc, J)
solver = NonlinearVariationalSolver(problem)

file = File(output + "data/ch1.pvd")

In [ ]:
# map mesh vertices to solution DOFs
#####################################

dof_coordinates = ME.tabulate_dof_coordinates()
c_dofs = ME.sub(0).dofmap().dofs()

# get indicies of sorted result
dofs      = np.squeeze(dof_coordinates[c_dofs])
asc_order = np.argsort(dofs)

In [ ]:
# time stepping
#####################################

(t, T) = (0.0, 10)
(n, N) = (0, int(round(T/dt, 0)))

plt.figure(1)
plt.title("$u(t)$")
plt.xlabel("$x$")
labels = []

sol = {}
sol['x'] = dofs[asc_order]
sol['t'] = np.zeros((N,1))
sol['u'] = np.zeros((len(sol['x']), len(sol['t'])))

t0 = perf_counter()
while n < N:
    
    # compute current solution
    # solver.solve(problem, u.vector()) # Newton
    solver.solve() # PETSc
    
    # save output
    # file << (u.split()[0], t)
    sol['t'][n] = t
    sol['u'][:,n] = u.vector()[c_dofs][asc_order]
    
    # update previous solution
    u0.vector()[:] = u.vector()
    
    # plot 6 solution snapshots
    if n % round(N/6,0) == 0:
        plt.plot(dofs[asc_order], u.vector()[c_dofs][asc_order])
        labels.append(f"t = {t/T:.2f}")
    
    # print progress
    if n % round(N/10,0) == 0:
        print(f"{n} / {N}")
        
    # increment time
    t += dt
    n += 1
    
tf = perf_counter()
print(f"\nTook {tf-t0:.2f} seconds.")

plt.legend(labels)
plt.show()

In [ ]:
def snapshot(i):
    plt.clf()
    plt.plot(sol['x'], sol['u'][:,i],'b-',linewidth = 3)       
    plt.rc('xtick',labelsize=12)
    plt.rc('ytick',labelsize=12)
    plt.ylim([-1.05, 1.05])
    plt.xlabel('$x$',fontsize='x-large')
    plt.ylabel('$u(t,x)$',fontsize='x-large')
    plt.title('Cahn-Hilliard')

In [ ]:
fig = plt.figure(figsize=(8,8), dpi=100)
snapshot(N-1)

In [ ]:
fig = plt.figure(figsize=(8,8), dpi=100)
anim = animation.FuncAnimation(fig,snapshot,frames=N)
if IC == RANDOM_ICs:
    gifname = 'cahn-hilliard-1d_rand.mp4'
elif IC == EXPR_ICs:
    gifname = 'cahn-hilliard-1d_expr.mp4'
anim.save(output + gifname,fps=N/10)